In [29]:
from ncp_potencia import *
from ncp_mantenimientos import *
from ncp_inputs import *
import datetime as dt
from tqdm import tqdm
import sys, os
import warnings
pd.set_option('display.max_rows', 10)
warnings.filterwarnings('ignore')

In [30]:
# Disable
def blockPrint():
    sys.stdout = open(os.devnull, 'w')

# Restore
def enablePrint():
    sys.stdout = sys.__stdout__

In [31]:
# blockPrint()

In [32]:
def get_cargas_parciales(start_date,end_date):
    print("Inicia get_cargas_parciales")
#     query = f"""
#     SELECT GTM_POE.FECHA,GTM_POE.generador_marginal 'UNIDAD MARGINAL',ROUND(GTM_POE.poe_usd_mwh,2) 'POE', CP.PRECIO_J1, CP.CARGA_J1, CP.PRECIO_J2, CP.CARGA_J2
#     FROM guatemala.GTM_POS_POE GTM_POE, (SELECT J1.FECHA,ROUND(J1.CVG,2) 'PRECIO_J1', J1.MWH 'CARGA_J1', ROUND(J2.CVG,2) 'PRECIO_J2', J2.MWH 'CARGA_J2'
#                                         FROM bi.GUA_CARGAS_PARCIALES_JEN J1, bi.GUA_CARGAS_PARCIALES_JEN J2
#                                         WHERE J1.generador = 'JEN-C1'
#                                         AND J2.generador = 'JEN-C2'
#                                         AND J1.FECHA = J2.FECHA) CP
#     WHERE
#           GTM_POE.FECHA = CP.FECHA
#     AND GTM_POE.FECHA >= STR_TO_DATE('{start_date.strftime('%Y-%m-%d')}','%%Y-%%m-%%d') AND GTM_POE.FECHA < STR_TO_DATE('{end_date.strftime('%Y-%m-%d')}','%%Y-%%m-%%d')
#     GROUP BY GTM_POE.FECHA
#     ORDER BY FECHA ASC;
#     """

    query = f"""
    SELECT GTM_POE.FECHA,GTM_POE.generador_marginal 'UNIDAD MARGINAL',ROUND(GTM_POE.poe,2) 'POE', CP.PRECIO_J1, CP.CARGA_J1, CP.PRECIO_J2, CP.CARGA_J2
    FROM guatemala.GTM_POS_POE_MENSUAL GTM_POE, (SELECT J1.FECHA,ROUND(J1.CVG,2) 'PRECIO_J1', J1.MWH 'CARGA_J1', ROUND(J2.CVG,2) 'PRECIO_J2', J2.MWH 'CARGA_J2'
                                        FROM bi.GUA_CARGAS_PARCIALES_JEN J1, bi.GUA_CARGAS_PARCIALES_JEN J2
                                        WHERE J1.generador = 'JEN-C1'
                                        AND J2.generador = 'JEN-C2'
                                        AND J1.FECHA = J2.FECHA) CP
    WHERE
          GTM_POE.FECHA = CP.FECHA
    AND GTM_POE.FECHA >= STR_TO_DATE('{start_date.strftime('%Y-%m-%d')}','%%Y-%%m-%%d') AND GTM_POE.FECHA < STR_TO_DATE('{end_date.strftime('%Y-%m-%d')}','%%Y-%%m-%%d')
    GROUP BY GTM_POE.FECHA
    ORDER BY FECHA ASC;
    """
    cp = mariaDB_DML("bi", query)
    print("el largo es",len(cp))
    print(cp)
    cp["datetime"] = cp["FECHA"].dt.strftime('%Y-%m-%d-%H')
    print("Termina get_cargas_parciales\n")
    return cp

In [33]:
def get_indisponibilidad(start_date, end_date, initial_hour, time_df):
    print("Inicia get_indisponibilidad")
    start_date = dt.datetime(start_date.year, start_date.month, start_date.day, int(initial_hour))  # fecha a realizar
    end_date = dt.datetime(end_date.year, end_date.month, end_date.day, int(initial_hour))
    ind = genera_masivo(start_date, end_date)
    ind = ind[ind["nemo"].str.contains("JEN")].rename(columns={"fecha":"date"})[["date","status", "nemo"]]
    ind["date"] = pd.to_datetime(ind["date"]).dt.strftime('%Y-%m-%d')
    ind_J1 = ind[ind["nemo"]=='JEN-C1'].rename(columns = {"status":"DISP_J1"}).drop("nemo",axis = 1)
    ind_J2 = ind[ind["nemo"]=='JEN-C2'].rename(columns = {"status":"DISP_J2"}).drop("nemo",axis = 1)
    result = pd.merge(time_df, ind_J1, how = "left", on="date")
    result = pd.merge(result, ind_J2, how = "left", on="date")
    result = result[["datetime", "DISP_J1", "DISP_J2"]]
    result[["DISP_J1","DISP_J2"]] = result[["DISP_J1","DISP_J2"]].fillna(True)
    print("Termina get_indisponibilidad\n")
    return result

In [34]:
def get_mantenimientos_df(start_date, end_date, time_df):
    print("Inicia get_mantenimientos_df")
    dias_ejecucion = (end_date - start_date).days 
    df_mantos = None
    hours = 48
    tipo = 'yhat'
#     for d in tqdm(range(0,dias_ejecucion,1)):
#         mantos_day = start_date + dt.timedelta(days = d)
#         d, x, y , n = get_demanda(mantos_day, hours, tipo)
#         m = mantenimiento(mantos_day, 8, y )
#         m["FECHA"] = pd.to_datetime(m["FECHA"])
#         m = m[(m["NEMO"].str.contains("JEN"))&(m["FECHA"].dt.date == mantos_day)]
#         df_mantos = m.copy() if (df_mantos is None) else pd.concat([df_mantos, m.copy()])
    
    df_mantos = pd.read_csv("GTM_MANTENIMIENTOS_NCP.csv").rename(columns = {"nemo_ncp":"NEMO", "mwh":"MWH"})
    
#     df_mantos["datetime"] = pd.to_datetime(df_mantos["FECHA"]).dt.strftime('%Y-%m-%d-%H')
    mantos_J1 = df_mantos[(df_mantos["NEMO"]=='JEN-C1')]
    mantos_J2 = df_mantos[(df_mantos["NEMO"]=='JEN-C2')]
    mantos_J1["MANTO_J1"] = False
    mantos_J2["MANTO_J2"] = False
    mantos_J1.loc[mantos_J1["MWH"]>0, "MANTO_J1"] = True
    mantos_J2.loc[mantos_J2["MWH"]>0, "MANTO_J2"] = True
    mantos_J1 = mantos_J1[["datetime","MANTO_J1"]]
    mantos_J2 = mantos_J2[["datetime","MANTO_J2"]]
    result = pd.merge(time_df, mantos_J1, how = "left", on="datetime")
    result = pd.merge(result, mantos_J2, how = "left", on="datetime")
    result = result[["datetime", "MANTO_J1", "MANTO_J2"]]
    result[["MANTO_J1","MANTO_J2"]] = result[["MANTO_J1","MANTO_J2"]].fillna(False)
    print("Termina get_mantenimientos_df\n")
    return result

In [35]:
def generate_base_table(start_date, end_date):
    time_df = pd.DataFrame(pd.date_range(start_date,end_date,freq="H")).rename(columns = {0:"datetime"})
    time_df["date"] = time_df["datetime"].dt.strftime('%Y-%m-%d')
    time_df["datetime"] = time_df["datetime"].dt.strftime('%Y-%m-%d-%H')
    cp = get_cargas_parciales(start_date,end_date)
    mant = get_mantenimientos_df(start_date, end_date, time_df)
    ind = get_indisponibilidad(start_date, end_date, 8, time_df)
    result = pd.merge(cp,ind,how="left",on="datetime")
    result = pd.merge(result,mant,how="left",on="datetime")
    result = result.drop("datetime", axis = 1)
    return result

In [36]:
def calculo_LIQ_POTENCIA(df):
    df["FECHA"] = pd.to_datetime(df["FECHA"])
    df["FECHA+1"] = df["FECHA"] + pd.DateOffset(months=1)
    df["HORAS"] = (df["FECHA+1"] - df["FECHA"]).dt.days.astype('int16')*24
    df = df.drop("FECHA+1", axis = 1)
    df["LIQ_POTENCIA"] = (df["POT_CONTRATADA"]*1000)/df["HORAS"]
    df["LIQ_POTENCIA"] = df["LIQ_POTENCIA"].round(2)
    return df

In [37]:
def complete_table(df):
    df["POT_CONTRATADA"] = 10
    df["POC"] = df["PRECIO_J1"]
    mask = df["CARGA_J2"]>df["CARGA_J1"]
    df.loc[mask, "POC"] = df[mask]["PRECIO_J2"]
    df["EJECUTA_OPCION"] = 0
    df.loc[df["POE"]>df["POC"], "EJECUTA_OPCION"] = 1
    df["LIQ_ENERGIA"] = 0
    df["DESC_POT"] = 0
    df.loc[(df["POE"]>df["POC"]) & ( ((df["CARGA_J1"]==0)&(df["MANTO_J1"]==False))|((df["CARGA_J2"]==0)&(df["MANTO_J2"]==False)) ),"DESC_POT"] = 1
    
    #===========
    df.loc[df["UNIDAD MARGINAL"].str.contains("JAGUAR"),"EJECUTA_OPCION"] = 0
    df.loc[(df["CARGA_J1"]== 0) & (df["CARGA_J2"]== 0),["DISP_J1", "DISP_J2"]] = False, False
#     print(len(df.loc[(df["DISP_J1"] is False)&(df["DISP_J2"] is False)&(df["MANTO_J1"] is False)&(df["MANTO_J2"] is False)& (df["CARGA_J1"]==0) & (df["CARGA_J2"]==0), ["EJECUTA_OPCION","DESC_POT" ]]))
    df.loc[(df["DISP_J1"] == False)&(df["DISP_J2"] == False)&(df["MANTO_J1"] == False)&(df["MANTO_J2"] == False)& (df["CARGA_J1"]==0) & (df["CARGA_J2"]==0), ["EJECUTA_OPCION","DESC_POT" ]] = 1, 0         
    #==========
    
    mask = df["EJECUTA_OPCION"] == 1
    df.loc[mask, "LIQ_ENERGIA"] = df[mask]["POE"] - df[mask]["POC"]
    df = calculo_LIQ_POTENCIA(df)

    

    
    df.loc[df["DESC_POT"]==1,"LIQ_POTENCIA"] *= 0.7
    df.loc[df["DESC_POT"]==1,"LIQ_ENERGIA"] = 0
    df["LIQ_POTENCIA"] = df["LIQ_POTENCIA"].round(2)
    
    df["LIQ_AMM"] = df["POE"]*df["POT_CONTRATADA"]
    df["LIQ_ENERGIA"] *= df["POT_CONTRATADA"]
    df["LIQ_POTENCIA"] *= df["POT_CONTRATADA"]
    df["SALDO_TOTAL"] = df["LIQ_POTENCIA"] + df["LIQ_AMM"] - df["LIQ_ENERGIA"] 
    df["SALDO_NETO"] = df["LIQ_AMM"] - df["SALDO_TOTAL"]
    #agregar campo de potencia contratada (en este caso 10 MW)
    return df

In [38]:
def generar_liquidacion(start_date, end_date):
    end_date += dt.timedelta(days = 1)
    base_table = generate_base_table(start_date, end_date)
    complete_liq = complete_table(base_table)
    return base_table, complete_liq

In [39]:
# liq_ene = generar_liquidacion(dt.date(2022,1,1), dt.date(2022,1,31))
# liq_oct.to_csv("liq_octubre.csv")

In [40]:
# def formatear_rango(cargas):
#     cargas["rango"]=cargas["rango"].str.replace('(', '', regex=True)
#     cargas["rango"]=cargas["rango"].str.replace(')', '', regex=True)
#     cargas["rango"]=cargas["rango"].str.replace('[', '', regex=True)
#     cargas["rango"]=cargas["rango"].str.replace(']', '', regex=True)
#     cargas["rango"]=cargas["rango"].str.replace(',', '', regex=True)
#     cargas["rango"]=cargas["rango"].str.replace('-', '', regex=True)
#     cargas["rango"]=cargas["rango"].str.replace('Pmax', '1000000', regex=True)
    
#     cargas_min=[]
#     cargas_max=[]
    
#     for i in range(0,len(cargas)):
#         linea=cargas["rango"].values[i].split()
#         cargas_min.append(float(linea[0]))
#         cargas_max.append(float(linea[1]))

#     cargas["min"]=cargas_min
#     cargas["max"]=cargas_max

#     return cargas

In [41]:
# def get_cargas_parciales(start_date,end_date):
#     file_name_cargas = "GTM_PRE_CARGAS_PARCIALES"
#     file_name_pos_carga_horaria = "GTM_POS_CARGA_HORARIA"

#     cargas = pd.read_csv(f"{file_name_cargas}.csv").rename(columns={"fecha":"datetime","nemo":"generador"}).fillna(0)
#     generacion = pd.read_csv(f"{file_name_pos_carga_horaria}.csv").rename(columns={"fecha":"datetime","valor":"MWH"}).fillna(0)

#     cargas=cargas.drop_duplicates(subset=["datetime","generador","rango"])
#     cargas=cargas.drop("fuente",axis=1)
#     cargas["datetime"]=pd.to_datetime(cargas["datetime"])
#     cargas["fecha"]=cargas["datetime"].dt.date
#     cargas=cargas.drop("datetime",axis=1)
#     cargas=formatear_rango(cargas)

#     generacion["datetime"]=pd.to_datetime(generacion["datetime"])
#     generacion["fecha"]=generacion["datetime"].dt.date
#     generacion["CVG"]=0
#     generacion["HORA"]=generacion["datetime"].dt.hour

#     join=pd.merge(cargas,generacion,how="left",on=["fecha","generador"])
#     join["CVG"]=0
#     join["HORA"]=join["datetime"].dt.hour

#     DR=join.drop("fecha",axis=1).copy() 
#     #donde DR corresponde al DATAFRAME RESULT
#     DR.loc[(DR["MWH"]>=DR["min"])&(DR["MWH"]<=DR["max"])&(DR["HORA"]>=0)&(DR["HORA"]<=5),"CVG"]=DR[(DR["MWH"]>=DR["min"])&(DR["MWH"]<=DR["max"])&(DR["HORA"]>=0)&(DR["HORA"]<=5)]["demanda_minima"]
#     DR.loc[(DR["MWH"]>=DR["min"])&(DR["MWH"]<=DR["max"])&(DR["HORA"]>=6)&(DR["HORA"]<=17),"CVG"]=DR[(DR["MWH"]>=DR["min"])&(DR["MWH"]<=DR["max"])&(DR["HORA"]>=6)&(DR["HORA"]<=17)]["demanda_media"]
#     DR.loc[(DR["MWH"]>=DR["min"])&(DR["MWH"]<=DR["max"])&(DR["HORA"]>=18)&(DR["HORA"]<=21),"CVG"]=DR[(DR["MWH"]>=DR["min"])&(DR["MWH"]<=DR["max"])&(DR["HORA"]>=18)&(DR["HORA"]<=21)]["demanda_maxima"]
#     DR.loc[(DR["MWH"]>=DR["min"])&(DR["MWH"]<=DR["max"])&(DR["HORA"]>=22)&(DR["HORA"]<=23),"CVG"]=DR[(DR["MWH"]>=DR["min"])&(DR["MWH"]<=DR["max"])&(DR["HORA"]>=22)&(DR["HORA"]<=23)]["demanda_minima"]
#     DR=DR[DR["CVG"]!=0]
#     DR=DR[["datetime","generador",'MWH','CVG']].sort_values(by=["datetime","generador"])
#     return DR

In [42]:
liq_nov = generar_liquidacion(dt.date(2022,11,1), dt.date(2022,11,30))
# liq_nov[1]
# liq_nov.to_csv("liq_noviembre.csv")

Inicia get_cargas_parciales

    SELECT GTM_POE.FECHA,GTM_POE.generador_marginal 'UNIDAD MARGINAL',ROUND(GTM_POE.poe,2) 'POE', CP.PRECIO_J1, CP.CARGA_J1, CP.PRECIO_J2, CP.CARGA_J2
    FROM guatemala.GTM_POS_POE_MENSUAL GTM_POE, (SELECT J1.FECHA,ROUND(J1.CVG,2) 'PRECIO_J1', J1.MWH 'CARGA_J1', ROUND(J2.CVG,2) 'PRECIO_J2', J2.MWH 'CARGA_J2'
                                        FROM bi.GUA_CARGAS_PARCIALES_JEN J1, bi.GUA_CARGAS_PARCIALES_JEN J2
                                        WHERE J1.generador = 'JEN-C1'
                                        AND J2.generador = 'JEN-C2'
                                        AND J1.FECHA = J2.FECHA) CP
    WHERE
          GTM_POE.FECHA = CP.FECHA
    AND GTM_POE.FECHA >= STR_TO_DATE('2022-11-01','%%Y-%%m-%%d') AND GTM_POE.FECHA < STR_TO_DATE('2022-12-01','%%Y-%%m-%%d')
    GROUP BY GTM_POE.FECHA
    ORDER BY FECHA ASC;
    
el largo es 720
                  FECHA                               UNIDAD MARGINAL    POE  \
0   2022-11-01 00:00:00 


            SELECT FECHA, GENERADOR, POTENCIA_PLANIFICADA, POTENCIA_REPLANIFICADA, POTENCIA_ACTIVA, POTENCIA_REACTIVA, ACTUALIZADA, REPLANIFICADA, CONDICION
            FROM GTM_CARGA_HORARIA WHERE DATE_FORMAT(FECHA, '%%Y-%%m-%%d %%H:%%i:%%s') >= '2022-10-31 00:00:00' and DATE_FORMAT(FECHA, '%%Y-%%m-%%d %%H:%%i:%%s') <= '2022-11-01 08:00:00'
            AND ACTUALIZADA = 'Si'
            AND GENERADOR IN ('JEN-C1', 'JEN-C2')
            ORDER BY GENERADOR,FECHA
        
Este es la unidad JEN-C1
                 FECHA GENERADOR  POTENCIA_ACTIVA ACTUALIZADA CONDICION
0  2022-10-31 00:00:00    JEN-C1            76.53          Si         -
1  2022-10-31 01:00:00    JEN-C1           103.21          Si         -
2  2022-10-31 02:00:00    JEN-C1           115.95          Si         -
3  2022-10-31 03:00:00    JEN-C1           119.87          Si         -
4  2022-10-31 04:00:00    JEN-C1           138.29          Si         -
..                 ...       ...              ...         ...      

Toda la columna contiene M,OD,Rra o FP, estado: on
Toda la columna contiene M,OD,Rra o FP, estado: on
Todos son nulos, estado: on
Todos son nulos, estado: on
Todos son nulos, estado: on
Todos son nulos, estado: on
Todos son nulos, estado: on
Todos son nulos, estado: on
Todos son nulos, estado: on
Toda la columna contiene IN, estado: off
Todos son nulos, estado: on
Toda la columna contiene M,OD,Rra o FP, estado: on
Todos son nulos, estado: on
Todos son nulos, estado: on
Toda la columna contiene M,OD,Rra o FP, estado: on
Todos son nulos, estado: on
Todos son nulos, estado: on
Toda la columna contiene M,OD,Rra o FP, estado: on
Todos son nulos, estado: on
Todos son nulos, estado: on
Todos son nulos, estado: on
Todos son nulos, estado: on
Todos son nulos, estado: on
Todos son nulos, estado: on
Todos son nulos, estado: on
Todos son nulos, estado: on
Todos son nulos, estado: on
Todos son nulos, estado: on
       nemo status_indi status_pre
0     LCU-E        True       True
1    SNT-E1       

Todos son nulos, estado: on
Todos son nulos, estado: on
Todos son nulos, estado: on
Todos son nulos, estado: on
Todos son nulos, estado: on
Todos son nulos, estado: on
Toda la columna contiene IN, estado: off
Todos son nulos, estado: on
Todos son nulos, estado: on
Todos son nulos, estado: on
Todos son nulos, estado: on
Todos son nulos, estado: on
Todos son nulos, estado: on
Todos son nulos, estado: on
Todos son nulos, estado: on
Todos son nulos, estado: on
Todos son nulos, estado: on
Toda la columna contiene M,OD,Rra o FP, estado: on
Todos son nulos, estado: on
Todos son nulos, estado: on
Todos son nulos, estado: on
Todos son nulos, estado: on
Todos son nulos, estado: on
Todos son nulos, estado: on
Todos son nulos, estado: on
       nemo status_indi status_pre
0     LCU-E        True       True
1    SNT-E1        True       True
2     VBL-E        True       True
3     CAL-G       False      False
4     ESC-V       False      False
..      ...         ...        ...
157  PQP-B7       F

Toda la columna contiene M,OD,Rra o FP, estado: on
Todos son nulos, estado: on
Todos son nulos, estado: on
Todos son nulos, estado: on
Todos son nulos, estado: on
Todos son nulos, estado: on
Toda la columna contiene M,OD,Rra o FP, estado: on
Todos son nulos, estado: on
Todos son nulos, estado: on
Toda la columna contiene M,OD,Rra o FP, estado: on
Todos son nulos, estado: on
Todos son nulos, estado: on
Todos son nulos, estado: on
Todos son nulos, estado: on
Todos son nulos, estado: on
Toda la columna contiene M,OD,Rra o FP, estado: on
Todos son nulos, estado: on
       nemo status_indi status_pre
0     LCU-E        True       True
1    SNT-E1        True       True
2     VBL-E        True       True
3     CAL-G       False      False
4     ESC-V       False      False
..      ...         ...        ...
159  PQP-B9       False      False
160  SAL-H2       False      False
161  SMA-H3       False      False
162  TER-B1       False      False
163  TER-B3       False      False

[164 rows x

...


            SELECT UNIDAD NEMO_IND,motivo MOTIVO_IND,sale_fecha S_FECHA_IND,entra_fecha E_FECHA_IND,potencia_mw MWH_IND
            from GTM_POS_INDISPONIBILIDADES
            where fecha = '2022-11-26 00:00:00'
            
Se ignoraron los siguientes movitivos en la disponibilidad ['MANT. PROG.' 'DEGRAD. DE CAPACIDAD']
       nemo  status
36   GAT-B1   False
37   GAT-B2   False
39   GCS-C1   False
63   HCV-H1   False
85   MAG-B3   False
..      ...     ...
168  RE2-H4   False
189  SIS-H1   False
190  SMA-H2   False
191  SMA-H3   False
192  STS-H2   False

[21 rows x 2 columns]

            SELECT fecha FECHA_D,nemo NEMO_D,Mwh MWH_D,descripcion DESCRIPCION_D,Fuente FROM GTM_PRE_PROGRAMA_DIARIO
            where fecha = '2022-11-28 00:00:00'
        
Esta es la ultima version  WEB28112022.xlsx
Todos son nulos, estado: on
Toda la columna contiene IN, estado: off
Todos son nulos, estado: on
Toda la columna contiene IN, estado: off
Toda la columna contiene IN, estado: off
Todos son nulos


            SELECT UNIDAD NEMO_IND,motivo MOTIVO_IND,sale_fecha S_FECHA_IND,entra_fecha E_FECHA_IND,potencia_mw MWH_IND
            from GTM_POS_INDISPONIBILIDADES
            where fecha = '2022-11-27 00:00:00'
            
Se ignoraron los siguientes movitivos en la disponibilidad ['MANT. PROG.' 'DEGRAD. DE CAPACIDAD']
       nemo  status
39   GAT-B1   False
40   GAT-B2   False
41   GCS-C1   False
66   HCV-H1   False
127  MTI-B1   False
..      ...     ...
225  SNT-E6   False
226  SNT-E7   False
227  SNT-E8   False
228  SNT-E9   False
229  STS-H2   False

[36 rows x 2 columns]

            SELECT fecha FECHA_D,nemo NEMO_D,Mwh MWH_D,descripcion DESCRIPCION_D,Fuente FROM GTM_PRE_PROGRAMA_DIARIO
            where fecha = '2022-11-29 00:00:00'
        
Esta es la ultima version  WEB29112022V1.xlsx
Todos son nulos, estado: on
Toda la columna contiene IN, estado: off
Todos son nulos, estado: on
Toda la columna contiene IN, estado: off
Toda la columna contiene IN, estado: off
Todos son nul


            SELECT UNIDAD NEMO_IND,motivo MOTIVO_IND,sale_fecha S_FECHA_IND,entra_fecha E_FECHA_IND,potencia_mw MWH_IND
            from GTM_POS_INDISPONIBILIDADES
            where fecha = '2022-11-28 00:00:00'
            
Se ignoraron los siguientes movitivos en la disponibilidad ['MANT. PROG.' 'DEGRAD. DE CAPACIDAD']
       nemo  status
3    ARI-O1   False
45   EPI-B3   False
49   GAT-B1   False
50   GAT-B2   False
52   GAT-B3   False
..      ...     ...
286  SNT-E8   False
287  SNT-E9   False
288  STS-H2   False
329  TUL-B1   False
338  TUL-B4   False

[46 rows x 2 columns]

            SELECT fecha FECHA_D,nemo NEMO_D,Mwh MWH_D,descripcion DESCRIPCION_D,Fuente FROM GTM_PRE_PROGRAMA_DIARIO
            where fecha = '2022-11-30 00:00:00'
        
Esta es la ultima version  WEB30112022.xlsx
Todos son nulos, estado: on
Todos son nulos, estado: on
Todos son nulos, estado: on
Toda la columna contiene IN, estado: off
Toda la columna contiene IN, estado: off
Todos son nulos, estado: on



            SELECT UNIDAD NEMO_IND,motivo MOTIVO_IND,sale_fecha S_FECHA_IND,entra_fecha E_FECHA_IND,potencia_mw MWH_IND
            from GTM_POS_INDISPONIBILIDADES
            where fecha = '2022-11-29 00:00:00'
            
Se ignoraron los siguientes movitivos en la disponibilidad ['MANT. PROG.' 'DEGRAD. DE CAPACIDAD']
       nemo  status
0    ARI-O1   False
29   GAT-B1   False
30   GAT-B2   False
31   GAT-B3   False
32   GCS-C1   False
..      ...     ...
209  SMA-H2   False
210  SMA-H3   False
228  STS-H2   False
289  TUL-B1   False
293  TUL-B4   False

[24 rows x 2 columns]

            SELECT fecha FECHA_D,nemo NEMO_D,Mwh MWH_D,descripcion DESCRIPCION_D,Fuente FROM GTM_PRE_PROGRAMA_DIARIO
            where fecha = '2022-12-01 00:00:00'
        
Esta es la ultima version  WEB01122022.xlsx
Todos son nulos, estado: on
Todos son nulos, estado: on
Todos son nulos, estado: on
Toda la columna contiene IN, estado: off
Toda la columna contiene IN, estado: off
Todos son nulos, estado: on



                select * from GTM_DICCIONARIO_NCP
            

               SELECT * FROM GTM_DICCIONARIO_CENTRALES
            

               SELECT MIX.nemo, GDN.nemo_ncp, MIX.tecnologia FROM GTM_DICCIONARIO_NCP GDN ,
                (SELECT GDC.id_central, GDU.id_unidad, GDU.nemo, GDC.tecnologia FROM GTM_DICCIONARIO_CENTRALES GDC, GTM_DICCIONARIO_UNIDADES GDU
                WHERE GDC.id_central = GDU.id_central) MIX
                WHERE
                (MIX.id_unidad = GDN.id_unidad AND GDN.id_unidad is not null )
                OR (MIX.id_central = GDN.id_central AND GDN.id_unidad is null)
            
Termina get_indisponibilidad



In [43]:
# liq_nov[1].to_csv("liq_noviembre.csv")

In [44]:
liq_nov[1]#.to_csv("liquidacion_noviembre_5.csv")

,FECHA,UNIDAD MARGINAL,POE,PRECIO_J1,CARGA_J1,PRECIO_J2,CARGA_J2,DISP_J1,DISP_J2,MANTO_J1,MANTO_J2,POT_CONTRATADA,POC,EJECUTA_OPCION,LIQ_ENERGIA,DESC_POT,HORAS,LIQ_POTENCIA,LIQ_AMM,SALDO_TOTAL,SALDO_NETO
0,2022-11-01 00:00:00,JAGUAR ENERGY U1,86.21,86.21,100.1,86.21,100.1,True,True,False,False,10,86.21,0,0.0,0,720,138.9,862.1,1001.0,-138.9
1,2022-11-01 01:00:00,JAGUAR ENERGY U1,92.20,92.20,87.0,86.21,100.1,True,True,False,False,10,86.21,0,0.0,0,720,138.9,922.0,1060.9,-138.9
2,2022-11-01 02:00:00,JAGUAR ENERGY U1,92.20,92.20,70.1,92.20,88.0,True,True,False,False,10,92.20,0,0.0,0,720,138.9,922.0,1060.9,-138.9
3,2022-11-01 03:00:00,JAGUAR ENERGY U1,92.20,92.20,70.1,92.20,70.1,True,True,False,False,10,92.20,0,0.0,0,720,138.9,922.0,1060.9,-138.9
4,2022-11-01 04:00:00,JAGUAR ENERGY U1,92.20,92.20,70.1,92.20,70.1,True,True,False,False,10,92.20,0,0.0,0,720,138.9,922.0,1060.9,-138.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
715,2022-11-30 19:00:00,CHIXOY,63.32,85.67,70.1,85.67,70.1,True,True,False,False,10,85.67,0,0.0,0,720,138.9,633.2,772.1,-138.9
716,2022-11-30 20:00:00,CHIXOY,63.32,85.67,70.1,85.67,70.1,True,True,False,False,10,85.67,0,0.0,0,720,138.9,633.2,772.1,-138.9
717,2022-11-30 21:00:00,INTERCONEXIÓN CON MÉXICO (IMPORTACIÓN EDC-I),62.68,85.67,70.1,85.67,70.1,True,True,False,False,10,85.67,0,0.0,0,720,138.9,626.8,765.7,-138.9
718,2022-11-30 22:00:00,SANTA ANA 2,39.62,85.07,70.1,85.07,70.1,True,True,False,False,10,85.07,0,0.0,0,720,138.9,396.2,535.1,-138.9


In [47]:
liq_nov[1].to_csv("liquidacion_noviembre_5.csv")

In [45]:
# dx = liq_nov[0].copy()
# dx = complete_liq = complete_table(dx)
# dx
# # dx[dx["UNIDAD MARGINAL"].str.contains("JAGUAR"),"EJECUTA_OPCION"] = 0
# # dx[(dx["CARGA_J1"]== 0) & (dx["CARGA_J2"]== 0),["DISP_J1", "DISP_J2"]] = False, False
# # dx[(dx["DISP_J1"] == False)&(dx["DISP_J2"] == False)&(dx["MANTO_J1"] == False)&(dx["MANTO_J2"] == False)& (dx["CARGA_J1"]==0) & (dx["CARGA_J2"]==0)]

In [46]:
# dx.to_csv("liquidacion_noviembre_4.csv")

No se ejecuta opcion si la unidad marginal es Jaguar en cualquiera de sus unidades

Si las dos unidades tienen cero, entonces disp es False. Entonces sí ejecuta opción pero no hay descuento de potencia.